# 日本語データセットをチャンクに分割

In [5]:
%pip install tiktoken langchain

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 21.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.9 MB/s eta 0:00:0000:0100:01
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:0000:01
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [6]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

## 1. 単一ファイルの動作確認

In [7]:
f = open('../data/sample-data.txt', 'r', encoding='UTF-8')

data = f.read()
print(data)

f.close()

導入
研究の目的と背景
現代の交通システムにおいて、環境負荷の低減とエネルギー効率の向上が重要な課題となっています。特に鉄道輸送においては、より持続可能な運転システムの開発が求められています。この文書は、水素ハイブリッド電車の技術とその展望について概説することを目的としています。

水素ハイブリッド電車の概要と定義
水素ハイブリッド電車は、鉄道輸送における持続可能なエネルギー源として注目を集めています。この鉄道車両は、燃料電池とバッテリーを組み合わせた駆動システムを利用しています。
燃料電池は、水素と酸素の化学反応により電力を発生するデバイスです。水素を燃料とし、酸素との反応によって電気エネルギーを生み出します。この反応によって発生する電力は、電気モーターを駆動するために使用されます。一方、バッテリーはエネルギーを貯蔵し、必要に応じて電力を供給する役割を果たします。バッテリーは、回生ブレーキングや低負荷時に発生する余剰エネルギーを貯蔵し、必要な時にモーターに電力を供給することで、効率的なエネルギー管理を実現します。
水素ハイブリッド電車の最大の利点の一つは、高いエネルギー効率と低い環境負荷です。燃料電池による発電は効率的であり、排出物としての水のみを生成します。これにより、大気汚染物質や温室効果ガスの排出を削減することができます。

また、水素ハイブリッド電車は長距離走行能力と短時間給水の特徴を持っています。水素のエネルギー密度が高いため、長距離を走行することが可能です。また、給水時間が短いため、効率的な運行スケジュールを維持することができます。

さらに、水素ハイブリッド電車は静音性と快適な乗り心地も提供します。燃料電池による駆動は非常に静かで、振動や騒音をほとんど発生しません。これにより、利用者にとって快適で静かな車内環境を提供することができます。

本論文では、まず水素ハイブリッド電車の技術要素について解説します。続いて、その利点やメリットを探求し、水素ハイブリッド電車が鉄道輸送にもたらす可能性を評価します。さらに、技術の課題や将来の展望についても考察します。最後に、水素ハイブリッド電車の技術が鉄道輸送に与える影響と、持続可能な交通システムへの展望について総括します。

1. 水素ハイブリッド電車の技術要素
燃料電池と水素供給
水素ハイブリッド電車の駆動システ

## ドキュメントのトークン数を計測
OpenAI モデルのトークン数を正確に数えるためには、[tiktoken](https://github.com/openai/tiktoken) ライブラリを利用します。

In [8]:
enc = tiktoken.get_encoding("gpt2")
print(len(enc.encode(data)))

11494


## テキストをチャンクに分割
通常 1 つのドキュメントに含まれるトークン数は 1 度のコンテキストに指定できる最大トークン数をゆうに超えます。今回はドキュメントの文を指定したトークン数以下のチャンクに分割します。

- テキストの分割方法<br>
RecursiveCharacterTextSplitter はチャンクが十分に小さくなるまで、順番に分割します。
- チャンクサイズの測定方法<br>
tiktoken ライブラリのトークナイザーを使用してトークン数を正確に測定します。

In [9]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='gpt2',
    chunk_size=500, 
    chunk_overlap=50
)

In [10]:
chunk = text_splitter.split_text(data)
chunk

['導入\n研究の目的と背景\n現代の交通システムにおいて、環境負荷の低減とエネルギー効率の向上が重要な課題となっています。特に鉄道輸送においては、より持続可能な運転システムの開発が求められています。この文書は、水素ハイブリッド電車の技術とその展望について概説することを目的としています。',
 '水素ハイブリッド電車の概要と定義\n水素ハイブリッド電車は、鉄道輸送における持続可能なエネルギー源として注目を集めています。この鉄道車両は、燃料電池とバッテリーを組み合わせた駆動システムを利用しています。\n燃料電池は、水素と酸素の化学反応により電力を発生するデバイスです。水素を燃料とし、酸素との反応によって電気エネルギーを生み出します。この反応によって発生する電力は、電気モーターを駆動するために使用されます。一方、バッテリーはエネルギーを貯蔵し、必要に応じて電力を供給する役割を果たします。バッテリーは、回生ブレーキングや低負荷時に発生する余剰エネルギーを貯蔵し、必要な時にモーターに電力を供給することで、効率的なエネルギー管理を実現します。',
 '水素ハイブリッド電車の最大の利点の一つは、高いエネルギー効率と低い環境負荷です。燃料電池による発電は効率的であり、排出物としての水のみを生成します。これにより、大気汚染物質や温室効果ガスの排出を削減することができます。',
 'また、水素ハイブリッド電車は長距離走行能力と短時間給水の特徴を持っています。水素のエネルギー密度が高いため、長距離を走行することが可能です。また、給水時間が短いため、効率的な運行スケジュールを維持することができます。\n\nさらに、水素ハイブリッド電車は静音性と快適な乗り心地も提供します。燃料電池による駆動は非常に静かで、振動や騒音をほとんど発生しません。これにより、利用者にとって快適で静かな車内環境を提供することができます。',
 '本論文では、まず水素ハイブリッド電車の技術要素について解説します。続いて、その利点やメリットを探求し、水素ハイブリッド電車が鉄道輸送にもたらす可能性を評価します。さらに、技術の課題や将来の展望についても考察します。最後に、水素ハイブリッド電車の技術が鉄道輸送に与える影響と、持続可能な交通システムへの展望について総括します。\n\n1. 水素ハイブリッド電車の技術要素\n燃料

In [11]:
#チャンク数
len(chunk)

36

In [12]:
#チャンク[0]の中身
chunk[0]

'導入\n研究の目的と背景\n現代の交通システムにおいて、環境負荷の低減とエネルギー効率の向上が重要な課題となっています。特に鉄道輸送においては、より持続可能な運転システムの開発が求められています。この文書は、水素ハイブリッド電車の技術とその展望について概説することを目的としています。'

In [13]:
#トークン数
print(len(enc.encode(chunk[0])))

215


In [14]:
#チャンクごとのトークン数確認
tokencounter = 0
for i, text in enumerate(chunk):
    print(i,len(enc.encode(text)))
    tokencounter = tokencounter + len(enc.encode(text))

print(tokencounter)

0 215
1 481
2 172
3 334
4 454
5 406
6 196
7 327
8 353
9 202
10 203
11 367
12 374
13 382
14 478
15 197
16 433
17 306
18 431
19 409
20 180
21 354
22 495
23 360
24 329
25 147
26 279
27 302
28 254
29 221
30 301
31 215
32 361
33 206
34 311
35 415
11450


## 2. ファイル一括処理
ドキュメントをチャンクに分割する方法を確認したら、実際に複数のデータセットを一括処理でチャンクごとのテキストファイルに分割します。

In [16]:
import os

enc = tiktoken.get_encoding("gpt2")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='gpt2',
    chunk_size=500, 
    chunk_overlap=50
)

#テキストファイルを読み込んで、指定のトークン数のチャンクファイルに分割します。
def splitChunkFile(filepath):
    f = open(filepath, 'r', encoding='UTF-8')
    data = f.read()
    chunk = text_splitter.split_text(data)

    #chunk単位でループ
    for i, chunkedtext in enumerate(chunk):
        
        dirname = os.path.dirname(filepath)
        basename = os.path.splitext(os.path.basename(filepath))[0]
        outputfilepath = dirname + "/output/" + basename + "-" + str(i) + ".txt"
        
        print(i, len(enc.encode(chunkedtext)), outputfilepath)
        with open(outputfilepath, 'w', encoding='UTF-8') as fo:
            fo.write(chunkedtext)

        fo.close()
    f.close()
   
    return

In [17]:
import glob
for p in glob.glob('../data/*.txt'):
    splitChunkFile(p)

0 307 ../data/output/20230502_to01-0.txt
1 390 ../data/output/20230502_to01-1.txt
2 449 ../data/output/20230502_to01-2.txt
3 459 ../data/output/20230502_to01-3.txt
4 492 ../data/output/20230502_to01-4.txt
5 498 ../data/output/20230502_to01-5.txt
6 416 ../data/output/20230502_to01-6.txt
7 473 ../data/output/20230502_to01-7.txt
0 483 ../data/output/20230508_a01-0.txt
1 459 ../data/output/20230508_a01-1.txt
2 497 ../data/output/20230508_a01-2.txt
3 499 ../data/output/20230508_a01-3.txt
4 194 ../data/output/20230508_a01-4.txt
0 443 ../data/output/20230508_c01-0.txt
1 497 ../data/output/20230508_c01-1.txt
2 497 ../data/output/20230508_c01-2.txt
3 450 ../data/output/20230508_c01-3.txt
4 499 ../data/output/20230508_c01-4.txt
5 23 ../data/output/20230508_c01-5.txt
0 497 ../data/output/20230508_hc001-0.txt
1 494 ../data/output/20230508_hc001-1.txt
2 478 ../data/output/20230508_hc001-2.txt
3 335 ../data/output/20230508_hc001-3.txt
0 490 ../data/output/20230508_ho01-0.txt
1 497 ../data/output/202